In [8]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr #data 받아오는 api 라이브러리
from pandas import json_normalize
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
import datetime as dt
from bs4 import BeautifulSoup
import traceback
import os
from keras import metrics
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

plt.style.use('bmh')

#Request URL API

In [9]:
cd /content/drive/My Drive/Colab Notebooks/KOSDAQ200

/content/drive/My Drive/Colab Notebooks/KOSDAQ200


In [11]:
import time
from datetime import timedelta

stock_data = pd.read_csv('KOSDAQ200.csv', encoding='CP949')
stock_data.종목코드 = stock_data.종목코드.map('{:06d}'.format) #이걸 해줘야 6자리 고정됨
stock_data = stock_data.rename(columns={'회사명':'name', '종목코드':'code'})

for idx,it in stock_data.iterrows():
  Training_start = time.time()
  filename = it[0] + ' ' + it[1] +'.csv'
  item_name = it[1]
  code = it[0]
  print(code, item_name)
    
  
  url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
  print("요청 URL = {}".format(url))
  res = requests.get(url)
  c = 0
  while res.reason == 'Internal Server Error':
    if c > 60:
      break;
    time.sleep(10)
    res = requests.get(url)
    c+=1

  res.encoding = 'utf-8'
  print(res)
  # 일자 데이터를 담을 df라는 DataFrame 정의 
  df = pd.DataFrame() 

  #페이지 크롤링 = 마지막 페이지 얻기
  soup = BeautifulSoup(res.text, 'lxml')
  el_table_navi = soup.find("table", class_="Nnavi")
  el_td_last = el_table_navi.find("td", class_="pgRR")
  pg_last = el_td_last.a.get('href').rsplit('&')[1]
  pg_last = pg_last.split('=')[1]
  pg_last = int(pg_last)

  if pg_last > 200:
    pg_last = 200 #8년동안 데이터
  print('마지막 쪽:', pg_last)

  for page in range(1, pg_last): #2170 items/124초 소요
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 

  df = df.dropna() 

  #csv 파일 만들기
  import csv
  if os.path.exists(filename):
    os.remove(filename)
  else:
    print("Create New {} file.".format(filename))

  try:
    with open (filename, 'w') as f:
      writer = csv.writer(f)
      for index, it in df.iterrows():
        writer.writerow(it)
  except:
    pass

  sec = time.time() - Training_start
  times = str(timedelta(seconds=sec)).split(".")
  print('Making Time: %s' %times[0])

069080 웹젠
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=069080
<Response [200]>
마지막 쪽: 200
Create New 069080 웹젠.csv file.
Making Time: 0:02:27
066970 엘앤에프
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=066970
<Response [200]>
마지막 쪽: 200
Create New 066970 엘앤에프.csv file.
Making Time: 0:02:24
064760 티씨케이
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=064760
<Response [200]>
마지막 쪽: 200
Create New 064760 티씨케이.csv file.
Making Time: 0:02:26
088800 에이스테크
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=088800
<Response [200]>
마지막 쪽: 200
Create New 088800 에이스테크.csv file.
Making Time: 0:02:23
022100 포스코 ICT
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=022100
<Response [200]>
마지막 쪽: 200
Create New 022100 포스코 ICT.csv file.
Making Time: 0:02:23
046890 서울반도체
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=046890
<Response [200]>
마지막 쪽: 200
Create New 046890 서울반도체.csv file.
Making Time: 0:02:24
085660 차바이오텍
요청 URL = http://finance.naver.com